player NBA2014-2015
วิภู สมบูรณ์ทรัพย์ 1600902439
โกเมศ เสือภูเขา 1600900607

In [ ]:
import numpy as np
import csv 
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as stats

from sqlalchemy import create_engine
from sklearn import linear_model
from sklearn.metrics import r2_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.cluster import AgglomerativeClustering
from sklearn.cluster import KMeans
from sklearn.cluster import SpectralClustering
from sklearn.mixture import GaussianMixture as GMM

import warnings
warnings.filterwarnings('ignore')

In [ ]:
%%time
NBA = pd.read_csv('/kaggle/input/nba-players-stats-20142015/players_stats.csv')

In [ ]:
NBAc = NBA[pd.notnull(NBA['BMI'])]

ตัวแปลทั้งหมด

In [ ]:
print(NBA.columns)

In [ ]:
duplicateRowsDF = NBA[NBA.duplicated()]
duplicateRowsDF

<br>
<b><center>Penjelasan Nama Kolom Diatas</center></b>
<table>
  <tr>
    <td><b>Name</b></td>
    <td>Nama Pemain</td>
    <td><b>Team</b></td>
    <td>Team Pemain</td>
    <td><b>Game Played</b></td>
    <td>Banyaknya Game yang Dimainkan</td>
    <td><b>MIN</b></td>
    <td>Menit Bermain</td>
    <td><b>PMPG</b></td>
    <td>Jumlah Menit Bermain Setiap Game</td>
  </tr>
  <tr>
    <td><b>EFF</b></td>
    <td>Effisiensi Pemain</td>
    <td><b>FGM</b></td>
    <td>Total Tembakan Masuk</td>
    <td><b>FGA</b></td>
    <td>Total Tembakan Bebas</td>
    <td><b>TFG</b></td>
    <td>Total Keseluruhan Tembakan</td>
    <td><b>PTS</b></td>
    <td>Point yang Disumbangkan Pemain</td>
  </tr>
</table>

Lakers are the champions!

In [ ]:
LAL_PLAYER = NBA.loc[NBA['Team']=='LAL']

LAL_PLAYER = LAL_PLAYER[['Name', 'Team', 'MIN', 'PTS', 'REB', 'AST', 'STL', 'BLK', 'TOV']]

LAL_PLAYER.index = range(LAL_PLAYER.shape[0])

t_min, t_pts, t_reb, t_ast, t_stl, t_blk, t_tov =  LAL_PLAYER[['MIN', 'PTS', 'REB', 'AST', 'STL', 'BLK', 'TOV']].sum()

attr_list = []

for i in LAL_PLAYER.index:
    
    p_name, p_team, p_min, p_pts, p_reb, p_ast, p_stl, p_blk, p_tov = LAL_PLAYER.iloc[i]
    
    attr_list.append([p_name, 'MIN', p_min/t_min])
    attr_list.append([p_name, 'PTS', p_pts/t_pts])
    attr_list.append([p_name, 'REB', p_reb/t_reb])
    attr_list.append([p_name, 'AST', p_ast/t_ast])
    attr_list.append([p_name, 'STL', p_stl/t_stl])
    attr_list.append([p_name, 'BLK', p_blk/t_blk])
    attr_list.append([p_name, 'TOV', p_tov/t_tov])
    
LAL_DATA = pd.DataFrame(attr_list, columns=['Name', 'Contribute_name', 'Values'])

In [ ]:
import altair as alt

bars = alt.Chart(LAL_DATA).mark_bar().encode(
    x=alt.X('sum(Values)', stack='zero'),
    y=alt.Y('Contribute_name'),
    color=alt.Color('Name')
)

text = alt.Chart(LAL_DATA).mark_text(dx=-15, dy=3, color='white').encode(
    x=alt.X('sum(Values):Q', stack='zero'),
    y=alt.Y('Contribute_name'),
    detail='Name',
    text=alt.Text('sum(Values):Q', format='.4f')
)

(bars + text).properties(width=800, height=300)

In [ ]:
NBA.head(10)

In [ ]:
NBA.Age.plot.hist(rwidth=0.90,color= "g");

In [ ]:
NBA.plot.scatter("Games Played", "PTS", alpha=0.5, color= "m");

status ในด้านต่างๆของplayer Lakers (MIN,PTS,REB,AST,STL,BLK,TOV)

In [ ]:
height_weight_info = NBA[['Name', 'Height', 'Weight', 'Pos']]

height_weight_info.head(10)

จำนวนplayer10คน เรียงตามตัวอักษร ชื่อ น้ำหนัก ส่วนสูงและตำแหล่งที่เล่น

In [ ]:
alt.Chart(height_weight_info).mark_circle(size=20).encode(
    x = 'Height',
    y = 'Weight',
    color = 'Pos',
    tooltip = ['Name', 'Height', 'Weight', 'Pos']
).properties(
    width=600, 
    height=600
).interactive()

เอาข้อมูลมาทำกราฟ สีของจุดคือบอกตำแหน่งว่าอยู่ในเกณ์ เราจะเห็นได้ว่า ส่วนสูงและน้ำหนักจะเฉลี่ยใกล้เคียงกันในแต่ละตำแหน่ง

การวิเคราะห์ 3P%
ทีมไหนทำได้ดีที่สุดในสามแต้ม 5ทีมแรก

In [ ]:
three_points_rate = NBA[['Team', '3P%']]

team_three_points_rate = three_points_rate.groupby('Team').mean()

team_three_points_rate['Team']  = team_three_points_rate.index

team_three_points_rate.index = [i for i in range(30)]

team_three_points_rate.head(10)

In [ ]:
team_data = alt.Chart(team_three_points_rate).mark_bar(
    color='lightblue'
).encode(
    x = 'Team',
    y = '3P%'
)
mean_rate = alt.Chart(team_three_points_rate).mark_rule(
    color='green'
).encode(
    y = 'mean(3P%)'
)
(team_data + mean_rate).properties(width=600)

แสดงข้อมูลเปอเซ็นต์3แต้มมาเป็นกราฟ

In [ ]:
three_points_rate = NBA[['Team', '3PA']]

team_three_points_rate = three_points_rate.groupby('Team').mean()

team_three_points_rate['Team']  = team_three_points_rate.index

team_three_points_rate.index = [i for i in range(30)]

team_three_points_rate.head(10)

In [ ]:
team_data = alt.Chart(team_three_points_rate).mark_bar(
    color='lightgreen'
).encode(
    x = 'Team',
    y = '3PA'
)
mean_rate = alt.Chart(team_three_points_rate).mark_rule(
    color='red'
).encode(
    y = 'mean(3PA)'
)
(team_data + mean_rate).properties(width=600)

In [ ]:
three_points_rate = NBA[['Team', '3PM']]

team_three_points_rate = three_points_rate.groupby('Team').mean()

team_three_points_rate['Team']  = team_three_points_rate.index

team_three_points_rate.index = [i for i in range(30)]

team_three_points_rate.head(10)

In [ ]:
team_data = alt.Chart(team_three_points_rate).mark_bar(
    color='lightyellow'
).encode(
    x = 'Team',
    y = '3PM'
)
mean_rate = alt.Chart(team_three_points_rate).mark_rule(
    color='red'
).encode(
    y = 'mean(3PM)'
)
(team_data + mean_rate).properties(width=600)

<img src='https://www.inquirer.com/resizer/qhSLJV2TtKQmV6I80DvKDREyXJE=/1400x932/smart/arc-anglerfish-arc2-prod-pmn.s3.amazonaws.com/public/4CUMLTQVCBDK5HG54LH2OPFZ5Q.jpg'>

In [ ]:
PTS_points_rate = NBA[['Team', 'PTS']]

team_PTS_points_rate = PTS_points_rate.groupby('Team').mean()

team_PTS_points_rate['Team']  = team_PTS_points_rate.index

team_PTS_points_rate.index = [i for i in range(30)]

team_PTS_points_rate.head(5)

In [ ]:
team_data = alt.Chart(team_PTS_points_rate).mark_bar(
    color='lightgreen'
).encode(
    x = 'Team',
    y = 'PTS'
)
mean_rate = alt.Chart(team_PTS_points_rate).mark_rule(
    color='red'
).encode(
    y = 'mean(PTS)'
)
(team_data + mean_rate).properties(width=600)

แต้มเฉลี่ยplayer for team

In [ ]:
NBA.plot.scatter

In [ ]:
NBA.plot.scatter("EFF", "PTS", alpha=0.5, color= "r", figsize=(13,5))
plt.xlabel('Efisiensi Pemain')
plt.ylabel('Point')

In [ ]:
msk = np.random.rand(len(NBAl)) < 0.8
train = NBAl[msk]
test = NBAl[~msk]

plt.figure(figsize=(13,5))
plt.scatter(train.EFF, train.PTS, alpha=0.5, color='blue')
plt.xlabel("Efisiensi Pemain")
plt.ylabel("Point")
plt.show()



In [ ]:
regr = linear_model.LinearRegression()
train_x = np.asanyarray(train[['EFF']])
train_y = np.asanyarray(train[['PTS']])
regr.fit (train_x, train_y)

print ('Coefficients: ', regr.coef_)
print ('Intercept: ',regr.intercept_)

In [ ]:
plt.figure(figsize=(13,5))
plt.scatter(train.EFF, train.PTS, alpha=0.5, color='blue')
plt.plot(train_x, regr.coef_[0][0]*train_x + regr.intercept_[0], '-r')
plt.xlabel("Efisiensi Pemain")
plt.ylabel("Point")

เปรียบเที่บยประสิธิภาพการเล่นกับแต่มที่ได้
EFF หรือที่เรียกว่าประสิทธิภาพของผู้เล่นในการเล่นบาสเก็ตบอลมีความสัมพันธ์กับคะแนนที่ได้รับเนื่องจากจากการวิเคราะห์อย่างง่ายของฉันบนเว็บไซต์สถิติทีม ESPN ทีมที่มีประสิทธิภาพในการโจมตีสูงสุด

In [ ]:
Boston = NBA.Team == 'BOS'
Detroit = NBA.Team == 'DET'
Cleveland = NBA.Team == 'CLE'
Orlando = NBA.Team == 'ORL'
Toronto = NBA.Team == 'TOR'
Washington = NBA.Team == 'WAS'
Philadelphia = NBA.Team == 'PHI'
Milwaukee = NBA.Team == 'MIL'
NewYork = NBA.Team == 'NYK'
Atlanta = NBA.Team == 'ATL'
Charlote = NBA.Team == 'CHA'
Chicago = NBA.Team == 'CHI'
Indiana = NBA.Team == 'IND'
Miami = NBA.Team == 'MIA'

dfeast = NBA[Boston | Detroit | Cleveland | Orlando | Toronto | Washington | Philadelphia | 
            Milwaukee | NewYork | Atlanta | Charlote | Chicago | Indiana | Miami]

In [ ]:
maxt = dfeast.groupby(['Team'])['PTS'].transform(max) == dfeast['PTS']
dfmax = dfeast[maxt]
dfmax.sort_values("EFF", ascending=False)

In [ ]:
import plotly.offline as py
import plotly.graph_objs as go

tr1 = go.Bar(
                 x = dfmax['Name'],
                 y = dfmax['EFF'],
                 name = 'Effisiensi',
                 marker = dict(color='crimson',
                              line = dict(color='rgba(0,0,0)', width=1)),
                 text = dfmax.Team)

tr2 = go.Bar(
                 x = dfmax['Name'],
                 y = dfmax['PTS'],
                 name = 'Points',
                 marker = dict(color='rgba(0, 1, 255, 1)',
                              line = dict(color='rgba(0,0,0)', width=1)),
                 text = dfmax.Team)
dn = [tr1, tr2]
layoutnew = go.Layout(barmode='group', title='Effisiensi Tertinggi Pemain pada TIM Regional Timur Terhadap Points')
fig = go.Figure(data=dn, layout=layoutnew)
fig.update_layout(barmode='stack')
py.iplot(fig)

ประสิทธิภาพของผู้เล่นที่สูงที่สุดในทีมภูมิภาคตะวันออกเทียบกับคะแนน

หากประเด็นข้างต้นเป็นประสิทธิภาพสูงสุดของผู้เล่นในทีมภูมิภาคตะวันออกฉันจะขยายช่วงหรือระยะทางตามผู้เล่นทั้งหมดในแต่ละทีมเพื่อวิเคราะห์การเปรียบเทียบทีมที่มีคะแนนมากกับจำนวนประสิทธิภาพที่ทำได้

In [ ]:
engine= create_engine('sqlite:///:memory:')

dfeast.to_sql('data_table', engine) 
dfeastrt= pd.read_sql_query('SELECT SUM("EFF"), Team FROM data_table group by Team', engine)
K = dfeastrt['SUM("EFF")']

In [ ]:
engine= create_engine('sqlite:///:memory:')

dfeast.to_sql('data_table', engine) 
dfeastrp= pd.read_sql_query('SELECT SUM("PTS"), Team FROM data_table group by Team', engine)

In [ ]:
dfeastrp.insert(1, 'SEFF', K)
dfeastrp.rename(
    columns={
        'SUM("PTS")': "SPTS"
    },
    inplace=True)

In [ ]:
dfeastrp['Rank'] = dfeastrp['SPTS'].rank(method='dense', ascending=False)
dfeastrp.sort_values("SPTS", ascending=False)

In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=dfeastrp['Team'], y=dfeastrp['SEFF'], fill='tozeroy',name = 'Total Team Efficiency'))
fig.add_trace(go.Scatter(x=dfeastrp['Team'], y=dfeastrp['SPTS'], fill='tonexty',name = 'Number of Team Points'))

fig.show()

เราก็สามารถวิเคราะห์ได้ว่ามีคะแนนเปอร์เซ็นเป็นกี่เปอร์เซ็นต์เพื่อให้ทีมอันดับหนึ่งในภูมิภาคนั้น ต่อมาจะเห็นได้ว่าแต่ละทีมจะต้องทำอย่างไรให้มีประสิทธิภาพมากขึ้น

In [ ]:
engine= create_engine('sqlite:///:memory:')

dfeast.to_sql('data_table', engine) 
gp= pd.read_sql_query('SELECT "Games Played" FROM data_table group by Team', engine)


In [ ]:
dfeastrp.insert(3, 'GP', gp)

In [ ]:
dfeastgpp = dfeastrp['SPTS'] / 82
dfeastgpp
# dfeastrp['GP'].max()

In [ ]:
fig = {
        'data': [ 
             {
                'values' : dfeastgpp,
                'labels' : dfeastrp['Rank'],
                'domain' : {'x': [0, 1]},
                'name' : 'Points / Game',
                'hoverinfo' : 'label+percent+name',
                'hole' : 0.3,
                'type' : 'pie'
              },
             ],
         'layout' : {
                     'title' : 'คะแนนเฉลี่ยต่อเกมที่ได้รับจากทีมตามการจัดอันดับ',
                     'annotations' : [
                                        { 'font' : {'size' : 20},
                                          'showarrow' : False,
                                          'text' : ' ',
                                          'x' : 0.20,
                                          'y' : 1
                                         },
                                      ]    
                     }
        }
py.iplot(fig)

In [ ]:
dfatl = dfeast[dfeast['Team'] == "ATL"]
dfatl

ประสิทธิภาพของผู้เล่นในทีมที่สูงขึ้นก็สามารถช่วยให้ทีมอยู่ในอันดับต้น ๆ ของภูมิภาคได้ ตัวอย่างเช่นทีม Atlanta อยู่ในอันดับที่หนึ่งในภูมิภาคตะวันออกเพราะมีคะแนนมากที่สุด

_________________________________________________________________________________________

การวิเคราะห์การจัดกลุ่มนี้มีวัตถุประสงค์เพื่อจำแนกตำแหน่งของผู้เล่นในฤดูกาลถัดไปตามสถานะบุคคลที่เกี่ยวข้องกับตำแหน่งของเขาในฐานะนักบาสเกตบอล
SG : Shooting Guard
PF : Power Forward
PG : Point Guard
C  : Center
SF : Small Forward

In [ ]:
NBAdel1 =NBAc.Name != 'Sim Bhullar'
NBAdel2 = NBAc.Name != 'Jerrelle Benimon'
NBAcl = NBAc[NBAdel1 & NBAdel2]

In [ ]:
NBAcl["TRB/MIN"] = NBAcl["REB"]/NBAcl["MIN"] 
NBAcl["AST/MIN"] = NBAcl["AST"]/NBAcl["MIN"]

fig, ax = plt.subplots()

x_var="AST/MIN"
y_var="TRB/MIN"

colors = {'SG':'blue', 'PF':'red', 'PG':'green', 'C':'purple', 'SF':'orange'}

ax.scatter(NBAcl[x_var], NBAcl[y_var], c=NBAcl['Pos'].apply(lambda x: colors[x]), s = 10)

# set a title and labels
ax.set_title('NBA Dataset')
ax.set_xlabel(x_var)
ax.set_ylabel(y_var)

In [ ]:
NBAn = NBAcl[["AST/MIN","TRB/MIN"]]

kmeans = KMeans(n_clusters = 5, init = 'k-means++', max_iter = 500, n_init = 10, random_state = 0)
y_kmeans = kmeans.fit_predict(NBAn)
print(kmeans.cluster_centers_)

In [ ]:
d0=NBAn[y_kmeans == 0]
d1=NBAn[y_kmeans == 1]
d2=NBAn[y_kmeans == 2]
d3=NBAn[y_kmeans == 3]
d4=NBAn[y_kmeans == 4]

plt.scatter(d0[x_var], d0[y_var], s = 10, c = 'blue', label = 'D0')
plt.scatter(d1[x_var], d1[y_var], s = 10, c = 'green', label = 'D1')
plt.scatter(d2[x_var], d2[y_var], s = 10, c = 'red', label = 'D2')
plt.scatter(d3[x_var], d3[y_var], s = 10, c = 'purple', label = 'D3')
plt.scatter(d4[x_var], d4[y_var], s = 10, c = 'orange', label = 'D4')

plt.scatter(kmeans.cluster_centers_[:, 0], kmeans.cluster_centers_[:,1], s = 100, c = 'yellow', label = 'Centroids')

In [ ]:
d0[x_var]='SG'
d1[x_var]='PF'
d2[x_var]='PG'
d3[x_var]='C'
d4[x_var]='SF'

NBAlist = pd.concat([d0[x_var], d1[x_var], d2[x_var], d3[x_var], d4[x_var]])

In [ ]:
NBAcluster = (NBAc[["Name", "Team", "Pos"]])
NBAcluster

NBAcl["TRB/MIN"]
NBAcl["AST/MIN"]

NBAcluster.insert(2, 'TRBMIN', NBAcl["TRB/MIN"])
NBAcluster.insert(3, 'ASTMIN', NBAcl["AST/MIN"])
NBAcluster.insert(5, 'Next Pos', NBAlist)
NBAcluster